In [1]:
import numpy as np 
import pandas as pd
from datetime import datetime

In [2]:
data_1min = pd.read_csv('AAPL_full_1min_adjsplit.csv')
data_1min.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
data_1min['Date'] = pd.to_datetime(data_1min['Date'], format="%Y-%m-%d %H:%M:%S")
data_1hour = pd.read_csv('AAPL_full_1hour_adjsplit.csv')
data_1hour.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
data_1min["Minute Log Return"] = np.log(data_1min['Close']).diff()

In [3]:
import pandas as pd

def calculate_hourly_volatility(dataframe):
   

    if 'Date' in dataframe.columns :
        dataframe['Date'] = pd.to_datetime(dataframe['Date'], format="%Y-%m-%d %H:%M:%S")

    
    date_log_return_df = dataframe[['Date', 'Minute Log Return']]

    date_log_return_df.set_index('Date', inplace=True)
   
    hourly_volatility = date_log_return_df.resample('H').apply(lambda x: (x ** 2).sum(axis=0)).apply(lambda x: x ** 0.5)
    hourly_volatility.reset_index(inplace=True)
    hourly_volatility.columns = ['Date', 'Hourly_Volatility']

    return hourly_volatility


result_df = calculate_hourly_volatility(data_1min)

In [4]:
def remove_zero_volatility(dataframe):

   
    filtered_dataframe = dataframe[dataframe['Hourly_Volatility'] != 0]

    return filtered_dataframe


filtered_result_df = remove_zero_volatility(result_df)

print(filtered_result_df)
newresults = filtered_result_df

                      Date  Hourly_Volatility
0      2005-01-03 08:00:00           0.010816
1      2005-01-03 09:00:00           0.018272
2      2005-01-03 10:00:00           0.010737
3      2005-01-03 11:00:00           0.005680
4      2005-01-03 12:00:00           0.004975
...                    ...                ...
167887 2024-02-28 15:00:00           0.002420
167888 2024-02-28 16:00:00           0.002219
167889 2024-02-28 17:00:00           0.002406
167890 2024-02-28 18:00:00           0.001517
167891 2024-02-28 19:00:00           0.001255

[72397 rows x 2 columns]


In [6]:
import pandas as pd

def matching_dataframes_bydate(df1, df2):

    df1['Date'] = pd.to_datetime(df1['Date'], errors='coerce')
    df2['Date'] = pd.to_datetime(df2['Date'], errors='coerce')

    common_dates = pd.merge(df1[['Date']], df2[['Date']], on='Date', how='inner')
    
    df1_sync = df1[df1['Date'].isin(common_dates['Date'])]
    df2_sync = df2[df2['Date'].isin(common_dates['Date'])]

    return df1_sync, df2_sync

newresults_sync, data_1hour_sync = matching_dataframes_bydate(newresults, data_1hour)


/var/folders/cb/9kkmszhd7fx9_93t2qb1_s640000gn/T/ipykernel_46636/234340229.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Date'] = pd.to_datetime(df1['Date'], errors='coerce')


In [7]:
data_1hour_sync['Date'] = pd.to_datetime(data_1hour_sync['Date'])
newresults_sync['Date'] = pd.to_datetime(newresults_sync['Date'])
data_1hour_sync.set_index('Date', inplace=True)
newresults_sync.set_index('Date', inplace=True)

/var/folders/cb/9kkmszhd7fx9_93t2qb1_s640000gn/T/ipykernel_46636/2225684696.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_1hour_sync['Date'] = pd.to_datetime(data_1hour_sync['Date'])
/var/folders/cb/9kkmszhd7fx9_93t2qb1_s640000gn/T/ipykernel_46636/2225684696.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newresults_sync['Date'] = pd.to_datetime(newresults_sync['Date'])


In [8]:
data_1hour_sync["Hourly Volatility"] = newresults_sync["Hourly_Volatility"]

/var/folders/cb/9kkmszhd7fx9_93t2qb1_s640000gn/T/ipykernel_46636/2337339055.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_1hour_sync["Hourly Volatility"] = newresults_sync["Hourly_Volatility"]


In [9]:
data_1hour_sync['Range'] = data_1hour_sync['High'] - data_1hour_sync['Low']
data_1hour_sync['Return'] = np.log(data_1hour_sync['Close']).diff()
data_1hour_sync['Return_Squared'] = np.square(data_1hour_sync['Return'])
data_1hour_sync = data_1hour_sync.dropna()

/var/folders/cb/9kkmszhd7fx9_93t2qb1_s640000gn/T/ipykernel_46636/68887939.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_1hour_sync['Range'] = data_1hour_sync['High'] - data_1hour_sync['Low']
/var/folders/cb/9kkmszhd7fx9_93t2qb1_s640000gn/T/ipykernel_46636/68887939.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_1hour_sync['Return'] = np.log(data_1hour_sync['Close']).diff()
/var/folders/cb/9kkmszhd7fx9_93t2qb1_s640000gn/T/ipykernel_46636/68887939.py:3: SettingWithCopyWarning: 
A value is 

In [10]:
pip install pandas_ta

Note: you may need to restart the kernel to use updated packages.


In [11]:
data = data_1hour_sync.dropna()
hourly_volatility = data["Hourly Volatility"]
df = pd.DataFrame({'hourly_volatility': hourly_volatility})
df['target'] = np.where(df['hourly_volatility'] > df['hourly_volatility'].shift(1), 1, 0)
df['target'].fillna(0, inplace=True)
data['target'] = df['target']
data_full = data.dropna()

In [12]:
data_full = data_full[["Return_Squared", "Hourly Volatility", "target"]]

In [33]:
data_full.to_csv(r"Intrahour Volatility Dataset.csv")